In [ ]:
import tweepy
import pandas as pd
import pyodbc
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#........import tools above........................

# consumer_key = "QIWkSvPlGKabO3NS4yVF13UJK"
# consumer_secret = "Xd3ZFgchNx0Txjo9fEVaedMzt3PSKKNRs8zHihZU1OxfBKWklR"
# access_token = "1389032855817183240-nkTj98q3URChTTUsFc2jPwMm4uKbOp"
# access_token_secret = "OeGehZwuxkGiquUGJIQ9iDzoHqaSUIYRjb8pWuDQC95ab"

consumer_key = "QIWkSvPlGKabO3NS4yVF13UJK" #input('Please enter consumer_key: ')
consumer_secret = "Xd3ZFgchNx0Txjo9fEVaedMzt3PSKKNRs8zHihZU1OxfBKWklR" #input('Please enter consumer_secret: ')
access_token = "1389032855817183240-nkTj98q3URChTTUsFc2jPwMm4uKbOp" #input('Please enter access_token: ')
access_token_secret = "OeGehZwuxkGiquUGJIQ9iDzoHqaSUIYRjb8pWuDQC95ab" #input('Please enter access_token_secret: ')
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
#.......set up finished above......................

def main():
    search_name_profile = input("Which user's profile you would like to check? (If you hope to check @JoeBiden, please enter JoeBiden here): ")
    search_name_followerinfo = input("Which user you would like to check his/her followers' info? (If you hope to check @JoeBiden, please enter JoeBiden here): ")
    followers_number = input(("How many {}'s followers you want to check? (please enter a number. For example: 1 or 20): ").format(search_name_followerinfo))
    search_word_1 = input("Plz enter the first word you want to check in Tweet: ")
    search_word_2 = input("Plz enter the second word you want to check in Tweet (If you only want to search 1 key word, press ENTER here): ")
    tweet_number = input("How many Tweet you want to check? (please enter a number. For example: 1 or 20): ")
    tweets_df = tweets_extractor(search_word_1, search_word_2, tweet_number)
    userinfo_df = userinfo_extractor(search_name_profile)
    followers_df = followerinfo_extractor(search_name_followerinfo, followers_number)
    get_all_tweets_df = get_all_tweets(search_name_profile)
    create_wordcloud(get_all_tweets_df)

    info_loader(tweets_df, userinfo_df, followers_df, get_all_tweets_df)


def tweets_extractor(search_word_1, search_word_2, tweet_number):
    try:
        tweets = tweepy.Cursor(api.search, q = (search_word_1, search_word_2)).items(int(tweet_number))
        tweets_list = [[tweet.created_at, tweet.user.screen_name, tweet.text, tweet.favorite_count] for tweet in tweets]
        tweets_df = pd.DataFrame(tweets_list, columns = ['Created_At', 'Screen_Name', 'Contents', 'Favorite_Count'])
        print(tweets_df)
        print("{} Tweets info extraction SUCCEED!!".format(tweet_number))       
    except:
        print("Tweets info extraction FAILED!!")
    return tweets_df
    

def userinfo_extractor(search_name_profile):
    try:
        userinfo = api.get_user(search_name_profile)
        userinfo_list = [userinfo.screen_name, userinfo.id, userinfo.description, userinfo.location]
        userinfo_df = pd.DataFrame(userinfo_list, index = ['User_Name', 'Id', 'Profile_Discription', 'Location'])
        userinfo_df = userinfo_df.transpose()
        print(userinfo_df)
        print("{}'s profile info extraction SUCCEED!!".format(search_name_profile))      
    except:
        print("User profile info extraction FAILED!!")
    return userinfo_df


def followerinfo_extractor(search_name_followerinfo, followers_number):
    try:
        followers = tweepy.Cursor(api.followers, screen_name = search_name_followerinfo).items(int(followers_number)) 
        followers_list = [[follower.screen_name, follower.id, follower.location] for follower in followers]
        followers_df = pd.DataFrame(followers_list, columns = ['User_Name', 'Id', 'Location']) 
        print(followers_df)      
        print("We have extracted {} follower's info from {}!SUCCEED!".format(followers_number, search_name_followerinfo))        
    except:
        print("Follower's info extraction FAILED!!")
    return followers_df


def get_all_tweets(search_name_profile):
    tweets = api.user_timeline(screen_name=search_name_profile, count=200, include_rts = False, tweet_mode = 'extended')
    all_tweets = []
    all_tweets.extend(tweets)
    oldest_id = tweets[-1].id
    while True:
        tweets = api.user_timeline(screen_name=search_name_profile, count=200, include_rts = False, max_id = oldest_id - 1, tweet_mode = 'extended')
        if len(tweets) == 0:
            break
        oldest_id = tweets[-1].id
        all_tweets.extend(tweets)
        
    outtweets = [[tweet.id_str, tweet.created_at, tweet.favorite_count, tweet.retweet_count, tweet.full_text.encode("utf-8").decode("utf-8")] for idx,tweet in enumerate(all_tweets)]
    df = pd.DataFrame(outtweets,columns=["Id","created_at","favorite_count","retweet_count", "text"])
    print(df)
    df.to_csv('%s_tweets_database.csv' % search_name_profile,index=True)
    return df

def create_wordcloud(get_all_tweets_df):
    allWords = ' '.join([twts for twts in get_all_tweets_df['text']])
    wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allWords)
    plt.imshow(wordCloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.show()

    
    
server = 'fatbetest.database.windows.net'
database = 'Twitter_Interim'
username = 'fatbe'
password = 'b61539275@bc'   
driver= '{ODBC Driver 17 for SQL Server}'

with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:

        def info_loader (tweets_df, userinfo_df, followers_df, get_all_tweets_df):
            cursor.execute('CREATE TABLE Tweets(Created_At datetime, Screen_Name nvarchar(200), Contents nvarchar(200), Favorite_Count int)')
            cursor.execute('CREATE TABLE UserProfile(User_Name nvarchar(20), Id int, Profile_Discription nvarchar(200), Location nvarchar(50))')
            cursor.execute('CREATE TABLE SocialNetwork_Info(User_Name nvarchar(20), Id nvarchar(50), Location nvarchar(50))')
            cursor.execute('CREATE TABLE get_all_tweets(Id nvarchar(200), created_at nvarchar(200), favorite_count nvarchar(200), retweet_count nvarchar(200), text nvarchar(max))')
            
            try:
                cursor.execute('DELETE FROM Tweets')
                for index,row in tweets_df.iterrows():
                    cursor.execute('INSERT INTO Tweets([Created_At],[Screen_Name],[Contents],[Favorite_Count]) values (?,?,?,?)', 
                                    row['Created_At'], 
                                    row['Screen_Name'], 
                                    row['Contents'],
                                    row['Favorite_Count'])
                print("tweets info loading to db SUCCEED!!")
            except:
                print("tweets info loading to db FAILED!!")

            try:
                cursor.execute('DELETE FROM UserProfile')
                for index,row in userinfo_df.iterrows():           
                    cursor.execute('INSERT INTO UserProfile([User_Name],[Id],[Profile_Discription],[Location]) values (?,?,?,?)', 
                                    row['User_Name'], 
                                    row['Id'], 
                                    row['Profile_Discription'],
                                    row['Location'])
                print("User info loading to db SUCCEED!!")
            except:
                print("user info loading to db FAILED!!")
                
            try:
                cursor.execute('DELETE FROM SocialNetwork_Info')
                for index,row in followers_df.iterrows():
                    cursor.execute('INSERT INTO SocialNetwork_Info([User_Name],[Id],[Location]) values (?,?,?)', 
                                    row['User_Name'], 
                                    row['Id'], 
                                    row['Location'])   
                print("Followers info loading to db SUCCEED!!") 
            except:
                print("Followers info loading to db FAILED!!")
                
                

            try:
                cursor.execute('DELETE FROM get_all_tweets')
                for index,row in get_all_tweets_df.iterrows():           
                    cursor.execute('INSERT INTO get_all_tweets([Id],[created_at],[favorite_count],[retweet_count],[text]) values (?,?,?,?,?)',                                     
                                    row['Id'], 
                                    row['created_at'],
                                    row['favorite_count'],
                                    row['retweet_count'],
                                    row['text'])
                print("get_all_tweets_df loading to db SUCCEED!!")
            except:
                print("get_all_tweets_df loading to db FAILED!!")
  

                    
            conn.commit()


if __name__ == "__main__":
    main()

ModuleNotFoundError: ignored